# P4. Aplicación en Tiempo Real

In [1]:
import policia_bender 
from policia_google import MotoMonitor
import cv2
from ultralytics import YOLO
import sys
import os
path_base = 'motos_detectadas/googleapp/'
key_path = os.path.join(path_base , 'config_google.json')
folder_id = '' # Folder Key ID Google Drive  - Almacenamiento
spreadsheet_id = "" # Sheet ID - Google sheet
sheet_name = "" # Sheetname
# Deshabilitar Monitor
# monitor = MotoMonitor(key_path, folder_id, spreadsheet_id, sheet_name)

# Función para deshabilitar temporalmente los prints
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')  # Redirige stdout a null

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stdout = self._original_stdout  # Restaura stdout

# Cargar el modelo entrenado
model = YOLO(r"D:\Proyectos_Python\Moto_Usuarios\runs\segment\1117SGD_0.01_11m_aumentation2\weights\best.pt", verbose=False)  # Ajusta la ruta si usas un modelo diferente
# Inicializar la cámaraa
nivel_confianza = 0.5
show_boxes = False
save_results =False
draw_box_object=False
target_size=(864, 480)
class_colors = {0: (0, 165, 255), 1: (255, 191, 0)}


In [2]:
# Uso de la función para una imagen local
image_path =  r"D:\Proyectos_Python\Moto_Usuarios\motos_detectadas_segmentadas\deteccion_20241116_130715_ORI.jpg"
image = cv2.imread(image_path)
CARGAR_IMAGEN_ORIGINAL = False

if image is None:
    print(f"Error al cargar la imagen: {image_path}")
# Redimensionar la imagen con padding
if not(CARGAR_IMAGEN_ORIGINAL):
    processed_image, (scale, x_offset, y_offset) = policia_bender.resize_with_aspect_and_padding(image, target_size)
    # Realizar predicción
    results = model(processed_image, conf=nivel_confianza)
    # Detectar motos y personas
    motos_con_personas = policia_bender.detect_people_on_motorbikes(results, threshold=nivel_confianza ,max_distance=30)

    #Notificar 
    image = policia_bender.draw_extended_boxes(image, motos_con_personas, scale, x_offset, y_offset)
    #monitor.procesar_moto_sospechosa(image, motos_con_personas) # <===========
    # Dibujar segmentaciones y cajas
    image = policia_bender.process_and_draw_segmentations(
        image, results, class_colors, scale, x_offset, y_offset, original_size=(image.shape[0], image.shape[1]) , draw_box_object=draw_box_object
    )

    # Dibujar las cajas extendidas
    
# Mostrar la imagen con detecciones
cv2.imshow("Detección en Imagen Local", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 480x864 2 motos, 3 personas, 38.6ms
Speed: 3.3ms preprocess, 38.6ms inference, 67.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 2
11.597170206688247
534.4670451435796
Distancia superada: 534.4670451435796
518.7863987900214
Distancia superada: 518.7863987900214
Cantidad de personas: 1
525.9719261865307
Distancia superada: 525.9719261865307
9.443981904839456
17.34419375229964
Cantidad de personas: 2


## Aplicacion de Video

Con el método process_video() te permite cargar un video en el cual puedes ejecutar el modelo, si defines una ruta de salida output_video_path, podrás guardar el video procesado.
Tambien peudes jugar con la maxima distancia definida y el nivel de confianza en las inferencias

In [4]:
from policia_google import MotoMonitor
import policia_bender
# monitor = MotoMonitor(key_path, folder_id, spreadsheet_id, sheet_name,intervalo_minimo= 1.5)
monitor = None

In [5]:
# Ruta de Video de Prueba
video_path = "motos_detectadas/video_prueba.mp4"
output_video_path = None
# Procesar el video
policia_bender.process_video(video_path, model, target_size=(864, 480), nivel_confianza=0.5, max_distance=50, output_path=output_video_path, monitor=monitor)



0: 480x864 1 moto, 1 persona, 211.3ms
Speed: 5.0ms preprocess, 211.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 1
13.177347913559233
Cantidad de personas: 1

0: 480x864 1 moto, 1 persona, 9.5ms
Speed: 3.0ms preprocess, 9.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 1
12.710493220095257
Cantidad de personas: 1

0: 480x864 1 moto, 1 persona, 10.5ms
Speed: 2.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 1
11.285585241624549
Cantidad de personas: 1

0: 480x864 1 moto, 1 persona, 17.5ms
Speed: 4.1ms preprocess, 17.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 1
12.010538283547334
Cantidad de personas: 1

0: 480x864 1 moto, 1 persona, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 1
14.394430780804193
Cantidad de personas: 1

0: 4

## Aplicación con Transmisión en Vivo

Usaremos una transmisión en video de Youtube, con el cual podremos ejecutar nuestro modelo con data completamente externa al contexto con el cual lo entrenamos!

In [7]:
#!pip install yt-dlp
#URL_LIVE = "https://www.youtube.com/live/3-SH_CMJsuk?si=g2Qnt0iNrQzsLr3m"
!yt-dlp -F https://www.youtube.com/watch?v=2TLnBhOWD2M&ab_channel=STREETSOFTHAILAND

[youtube] Extracting URL: https://www.youtube.com/watch?v=2TLnBhOWD2M
[youtube] 2TLnBhOWD2M: Downloading webpage
[youtube] 2TLnBhOWD2M: Downloading ios player API JSON
[youtube] 2TLnBhOWD2M: Downloading mweb player API JSON
[youtube] 2TLnBhOWD2M: Downloading m3u8 information
[youtube] 2TLnBhOWD2M: Downloading m3u8 information
[info] Available formats for 2TLnBhOWD2M:
ID  EXT RESOLUTION FPS |   TBR PROTO | VCODEC        VBR ACODEC     MORE INFO
-----------------------------------------------------------------------------
233 mp4 audio only     |       m3u8  | audio only        unknown    Default
234 mp4 audio only     |       m3u8  | audio only        unknown    Default
269 mp4 256x144     15 |  290k m3u8  | avc1.42C00B  290k video only
91  mp4 256x144     15 |  290k m3u8  | avc1.42c00b       mp4a.40.5
229 mp4 426x240     30 |  546k m3u8  | avc1.4D4015  546k video only
92  mp4 426x240     30 |  546k m3u8  | avc1.4d4015       mp4a.40.5
230 mp4 640x360     30 | 1210k m3u8  | avc1.4D401E 1

"ab_channel" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [8]:
URL_LIVE_YOUTUBE = !yt-dlp -f 96  -g https://www.youtube.com/watch?v=2TLnBhOWD2M&ab_channel=STREETSOFTHAILAND
URL_LIVE_YOUTUBE  = URL_LIVE_YOUTUBE[0]

URL_LIVE_YOUTUBE

'https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1733634934/ei/FtdUZ7qTOOiP-LAP0uPEgQY/ip/38.25.30.11/id/2TLnBhOWD2M.3/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr1---sn-fpoxuuxa-aphe.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/spc/qtApAUmfOTRt8rMtDm4ohzrCi68faORiCCumgaar9F9boGBiUEZO76_maK8HsRE/vprv/1/playlist_type/DVR/initcwndbps/1845000/met/1733613335,/mh/ob/mm/44/mn/sn-fpoxuuxa-aphe/ms/lva/mv/m/mvi/1/pl/24/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51326932,51331020,51335594,51347746/mt/1733612903/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,playlist_duration,manifest_duration,spc,vprv,playlist_type/sig/AJfQdSswRQIgb2tlsfRL7ZVl-srnaSTjzx09kkYGc1A21mustS4L6p8CIQD7-Fp-tTTjThsMqkV97B8XruyrgdxG1kWpBOP2lXUxaQ%3D%3D/lsparams/hls_chunk_host,initcwndbps,met,mh,mm,mn,ms,mv,mvi,pl

In [ ]:
import cv2
import time

# Reemplaza "URL_HLS_EXTRAIDA" con la URL del stream

# Inicializar el stream con OpenCV
cap = cv2.VideoCapture(URL_LIVE_YOUTUBE)

if not cap.isOpened():
    print("Error: No se pudo abrir el stream.")
else:
    print("Stream abierto con éxito. Presiona 'q' en la ventana para detener.")

    # Ajustar la tasa de fotogramas esperada
    fps = cap.get(cv2.CAP_PROP_FPS) or 30  # Usa el FPS reportado o un valor estándar
    delay = int(1000 / fps)

    while True:
        start_time = time.time()

        ret, image = cap.read()
        if not ret:
            print("Error al leer el frame del stream.")
            break
        processed_image, (scale, x_offset, y_offset) = policia_bender.resize_with_aspect_and_padding(image, target_size)
        # Realizar predicción
        results = model(processed_image, conf=nivel_confianza)
        # Detectar motos y personas
        motos_con_personas = policia_bender.detect_people_on_motorbikes(results, threshold=nivel_confianza ,max_distance=30)
    
        #Notificar 
        image = policia_bender.draw_extended_boxes(image, motos_con_personas, scale, x_offset, y_offset)
        #monitor.procesar_moto_sospechosa(image, motos_con_personas)  <====
        # Dibujar segmentaciones y cajas
        image = policia_bender.process_and_draw_segmentations(
            image, results, class_colors, scale, x_offset, y_offset, original_size=(image.shape[0], image.shape[1]) , draw_box_object=draw_box_object
        )
        # Mostrar el frame en una ventana independiente
        cv2.imshow("Stream en Vivo", image)

        # Presiona 'q' para salir
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Stream detenido por el usuario.")
            break

        # Asegurar sincronización con el framerate
        elapsed_time = time.time() - start_time
        if elapsed_time < (1 / fps):
            time.sleep((1 / fps) - elapsed_time)

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()


Stream abierto con éxito. Presiona 'q' en la ventana para detener.

0: 480x864 (no detections), 71.5ms
Speed: 3.0ms preprocess, 71.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 0

0: 480x864 (no detections), 10.6ms
Speed: 1.0ms preprocess, 10.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 0

0: 480x864 (no detections), 18.6ms
Speed: 1.0ms preprocess, 18.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 0

0: 480x864 (no detections), 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 0

0: 480x864 (no detections), 9.8ms
Speed: 1.0ms preprocess, 9.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 0

0: 480x864 (no detections), 9.7ms
Speed: 1.0ms preprocess, 9.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 864)
Cantidad de Motos: 0

0: 480x864 (